## Todo
1. Voxel-based interpolation, reproducing C++ results
2. Replace 0-prediction with knn
3. Experiment with single knn-hybrid approach

In [1]:
import os
import numpy as np
import open3d
import time
from utils.metric import ConfusionMatrix
from utils.point_cloud_util import load_labels, colorize_point_cloud
import multiprocessing

In [17]:
class LabelCounter:
    """
    A simple but non-optimized implementation
    """
    def __init__(self):
        self.labels = []
        
    def increment(self, label):
        self.labels.append(label)
        
    def get_label(self):
        return np.bincount(self.labels).argmax()
    
def get_voxel(point, voxel_size=0.1):
    """
    Returns a voxel tuple
    point: [x, y, z]
    """
    return tuple([np.floor(float(val) / voxel_size) + 0.5 for val in point])

In [3]:
sparse_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/sparse"
dense_dir = "/home/ylao/repo/Open3D-PointNet-Semantic/results/dense"
gt_dir = "/home/ylao/data/semantic3d"

In [4]:
file_prefix = "untermaederbrunnen_station1_xyz_intensity_rgb"

# Sparse labels
sparse_labels = np.array(load_labels(os.path.join(sparse_dir, file_prefix + ".labels")), dtype=np.int32)

# Sparse points
sparse_pcd = open3d.read_point_cloud(os.path.join(sparse_dir, file_prefix + ".pcd"))
sparse_points = np.asarray(sparse_pcd.points)

# Dense points
dense_pcd = open3d.read_point_cloud(os.path.join(gt_dir, file_prefix + ".pcd"))
dense_points = np.asarray(dense_pcd.points)

# Ground-trugh dense labels
gt_labels = load_labels(os.path.join(gt_dir, file_prefix + ".labels"))

In [ ]:
map_voxel_to_label_counter = dict()
for sparse_point, sparse_label in zip(sparse_points, sparse_labels):
    voxel = get_voxel(sparse_point)
    if voxel not in map_voxel_to_label_counter:
        map_voxel_to_label_counter[voxel] = LabelCounter()
    map_voxel_to_label_counter[voxel].increment(sparse_label)
print("{} sparse points, {} registered voxels".format(len(sparse_points), 
                                                      len(map_voxel_to_label_counter)))